[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/floleuerer/ml-tutorials/blob/main/06_tabular.ipynb)

# Vorbereitungen

Laufzeit-Typ ändern. Python-Pakete von fast.ai installieren und importieren.

In [ ]:
!pip install -Uq fastai

In [ ]:
from fastai.tabular.all import *

# Tabellarische Daten

Zur Veranschaulichung der tabellarischen Anwendung verwenden wir das Beispiel des Datensatzes für Erwachsene, in dem wir anhand einiger allgemeiner Daten vorhersagen müssen, ob eine Person mehr oder weniger als 50.000 USD pro Jahr verdient.

Wir können ein Beispiel dieses Datensatzes mit dem üblichen Befehl untar_data herunterladen:

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

Dann können wir uns ansehen, wie die Daten strukturiert sind:

In [ ]:
df = pd.read_csv(path/'adult.csv')
df.head()

Einige der Spalten sind fortlaufend (wie 'age') und wir werden sie als Gleitkommazahlen behandeln, die wir unserem Modell direkt zuführen können. Andere sind kategorisch (wie 'workclass' oder 'education') und wir konvertieren sie in einen eindeutigen Index, den wir als Input für einen `embedding layer` verwenden. Wir können unsere kategorialen und fortlaufenden Spaltennamen sowie den Namen der abhängigen Variablen in den Factory-Methoden von `TabularDataLoaders` angeben:

In [ ]:
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
    cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
    cont_names = ['age', 'fnlwgt', 'education-num'],
    procs = [Categorify, FillMissing, Normalize])

Der letzte Teil (`procs`) ist die Liste der `Preprocessors`, die wir auf unsere Daten anwenden:

- `Categorify` nimmt jede kategoriale Variable und erstellt eine Zuordnung von einer Ganzzahl zu eindeutigen Kategorien. Anschließend werden die Werte durch den entsprechenden Index ersetzt.
- `FillMissing` füllt die fehlenden Werte in den kontinuierlichen Variablen mit dem Median der vorhandenen Werte (Sie können einen bestimmten Wert auswählen, wenn Sie dies bevorzugen).
- `Normalize` normalisiert die stetigen Variablen (subtrahieren Sie den Mittelwert und dividieren Sie durch den Standard)

In [ ]:
dls.show_batch()

Das Trainieren eines Modells verhält sich wie bei den anderen Beispielen. Wir erstellen einen `Learner` und trainieren mit `learn.fit_one_cycle()` das Modell.

# Training

In [ ]:
learn = tabular_learner(dls, metrics=accuracy)

In [ ]:
learn.fit_one_cycle(3)

Mit `learn.show_results()` können wir uns die Ergebnisse anzeigen lassen. In der Spalte `salery_pred` sehen wir die vorhersage des Modells.

In [ ]:
learn.show_results(max_n=20)

`learn.predict()` macht einer Vorhersage für einen einzelnen Datensatz.

In [ ]:
row, clas, probs = learn.predict(df.iloc[10])

In [ ]:
row.show()